In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline

from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

from vecstack import StackingTransformer

import joblib

In [2]:
apart_sep = pd.read_csv('apart_all(2016-2021).csv') # 2016-2021 아파트 데이터 불러오기

In [3]:
apart_sep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493729 entries, 0 to 493728
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   자치구명    493729 non-null  object 
 1   법정동명    493729 non-null  object 
 2   신고년도    493729 non-null  int64  
 3   건물면적    493729 non-null  float64
 4   층정보     493729 non-null  int64  
 5   물건금액    493729 non-null  int64  
 6   건축년도    493729 non-null  int64  
 7   건물명     493729 non-null  object 
 8   경과년도    493729 non-null  int64  
 9   브랜드점수   493729 non-null  int64  
 10  매매지수    493729 non-null  float64
dtypes: float64(2), int64(6), object(3)
memory usage: 41.4+ MB


In [4]:
# ['노원구', '송파구', '강서구', '강남구', '강동구', '구로구', '성북구', '양천구', '도봉구', '서초구',
#        '영등포구', '성동구', '마포구', '동작구', '동대문구', '은평구', '중랑구', '서대문구', '관악구', '용산구',
#        '강북구', '광진구', '금천구', '중구', '종로구']

apart_sep['자치구명'].value_counts().keys() # 자치구 확인

Index(['노원구', '송파구', '강서구', '강남구', '강동구', '구로구', '성북구', '양천구', '도봉구', '서초구',
       '영등포구', '성동구', '마포구', '동작구', '동대문구', '은평구', '중랑구', '서대문구', '관악구', '용산구',
       '강북구', '광진구', '금천구', '중구', '종로구'],
      dtype='object')

In [5]:
# 구별 모델링을 위한 한글/영문 구 이름 Dictionary 생성
gu = {'노원구':'nowon', '송파구':'songpa', '강서구':'gangseo', '강남구':'gangnam', '강동구':'gangdong', '구로구':'guro', '성북구':'seongbuk', '양천구':'yangcheon', '도봉구':'dobong', '서초구':'seocho',
        '영등포구':'yeongdeungpo', '성동구':'sungdong', '마포구':'mapo', '동작구':'dongjak', '동대문구':'dongdaemoon', '은평구':'eunpyeong', '중랑구':'jungnang', '서대문구':'seodaemoon', '관악구':'gwanak', '용산구':'yongsan',
        '강북구':'gangbuk', '광진구':'gwangjin', '금천구':'geumcheon', '중구':'junggu', '종로구':'jongno'}

In [6]:
model_score = {} # 모델 성능지표 저장용 dict 생성

In [8]:
# 1. 모델 생성 
# 2. [model, pipeline, stack] pkl 파일 저장
# 3. 성능 평가 점수 저장

for kor, eng in gu.items():
    globals()[f'df_{eng}'] = apart_sep[apart_sep['자치구명']==kor] # df_nowon, df_gangnam ...
    
    x = globals()[f'df_{eng}'].drop(['물건금액'],axis=1)
    y = globals()[f'df_{eng}']['물건금액']
    
    globals()[f'df_{eng}'].to_csv(f'{kor}.csv', encoding='cp949', index=False) # 구별 csv 저장
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=123) # train, test 분리
    
    # 연속형 변수 열 입력
    numeric_features = ['매매지수','신고년도','건물면적','층정보','건축년도','경과년도']
    numeric_transformer = StandardScaler()

    # 범주형 변수 열 입력
    categorical_features = ['자치구명','법정동명','건물명','브랜드점수']
    categorical_transformer = OneHotEncoder(categories='auto', handle_unknown='ignore') # 범주형 데이터가 x_train, x_test에 고르게 들어가지 않는 경우 pass 하기 위해 handel_unknown param ='ignore' 로 설정

    preprocessor = ColumnTransformer(
        transformers=[ # List of (name, transformer, column(s))
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])
    
    preprocessor_pipe = Pipeline(steps=[('preprocessor', preprocessor)]) # 파이프라인 단계 입력
    
    preprocessor_pipe.fit(x_train, x_test) # 변수 변환
    
    # 변환된 변수로 transform
    x_train_transformed = preprocessor_pipe.transform(x_train)
    x_test_transformed = preprocessor_pipe.transform(x_test)
    
    # stacking 1 level model 지정
    estimators = [ 
    ('LightGBM', LGBMRegressor(learning_rate=0.05, n_estimators=300, num_leaves=100)),
    ('XGBoost', XGBRegressor(n_estimators=250)),
    ('RandomForest', RandomForestRegressor(n_estimators=200)),
    ('MLP', MLPRegressor(batch_size=50, hidden_layer_sizes=(128,256), learning_rate_init=0.05, max_iter=400))]
    
    # Initialize StackingTransformer
    stack = StackingTransformer(estimators, 
                                regression = True, 
                                metric = mean_squared_error, 
                                n_folds = 10, stratified = True, shuffle = True, 
                                random_state = 123, verbose = 0)
    
    # Fit
    stack = stack.fit(x_train_transformed, y_train)
    
    # Get your stacked features
    S_train = stack.transform(x_train_transformed)
    S_test = stack.transform(x_test_transformed)
    
    # Use 2nd level estimator with stacked features
    model = CatBoostRegressor(depth=10, l2_leaf_reg = 1, iterations=500, learning_rate = 0.05) 
    model = model.fit(S_train, y_train) 

    y_pred = model.predict(S_test) 
    
    # 모델 성능평가 점수를 구별로 list에 모아둔 뒤 dict로 저장 
    model_score_temp = []
    model_score_temp.append(f'{kor}')
    model_score_temp.append((f'RMSE : {(mean_squared_error(y_test, y_pred)**0.5):.1f}'))
    model_score_temp.append((f'MAE : {(mean_absolute_error(y_test, y_pred)**0.5):.1f}'))
    model_score_temp.append((f'R2 score : {(r2_score(y_test, y_pred)*100):.2f} %'))
    model_score[kor] = model_score_temp
    model_score_temp = [] # 성능평가 점수 초기화
    
    # 구별 pkl 파일 저장 [Model, Pipeline(feature scaling), stacking]
    joblib.dump(model, f'model_apart_{eng}.pkl', compress=True) # model save
    joblib.dump(preprocessor_pipe, f'pipeline_{eng}.pkl', compress=True) # feature scaling pipeline save
    joblib.dump(stack, f'stack_{eng}.pkl', compress=True) # stacking transform save

# 모델 성능 DataFrame화 후 csv로 저장
df2 = pd.DataFrame.from_dict(model_score, orient='index')
df2.to_csv('model_score(Stacking).csv', encoding='cp949', index=False)


C:\Users\meteo\.conda\envs\final\lib\site-packages\vecstack\coresk.py:382: UserWarning: This is regression task hence classification-specificparameters set to ``True`` were ignored: ``stratified``
  warnings.warn(warn_str, UserWarning)


0:	learn: 171642308.3847383	total: 195ms	remaining: 1m 37s
1:	learn: 163784722.6801116	total: 216ms	remaining: 53.9s
2:	learn: 156319775.6921202	total: 238ms	remaining: 39.4s
3:	learn: 149255696.6852976	total: 259ms	remaining: 32.1s
4:	learn: 142629071.6063773	total: 280ms	remaining: 27.7s
5:	learn: 136346117.6600122	total: 300ms	remaining: 24.7s
6:	learn: 130295518.4220944	total: 320ms	remaining: 22.6s
7:	learn: 124613897.6105853	total: 345ms	remaining: 21.2s
8:	learn: 119175071.4525921	total: 370ms	remaining: 20.2s
9:	learn: 114038102.2345876	total: 396ms	remaining: 19.4s
10:	learn: 109192250.6681261	total: 422ms	remaining: 18.8s
11:	learn: 104594939.6523016	total: 450ms	remaining: 18.3s
12:	learn: 100264778.3001644	total: 477ms	remaining: 17.9s
13:	learn: 96163329.5754990	total: 503ms	remaining: 17.5s
14:	learn: 92323097.5211963	total: 529ms	remaining: 17.1s
15:	learn: 88714688.3539542	total: 555ms	remaining: 16.8s
16:	learn: 85291833.7576904	total: 581ms	remaining: 16.5s
17:	learn:

C:\Users\meteo\.conda\envs\final\lib\site-packages\vecstack\coresk.py:382: UserWarning: This is regression task hence classification-specificparameters set to ``True`` were ignored: ``stratified``
  warnings.warn(warn_str, UserWarning)


0:	learn: 485420775.1820909	total: 20.3ms	remaining: 10.1s
1:	learn: 462764219.7901857	total: 40.8ms	remaining: 10.2s
2:	learn: 441225145.5518346	total: 60.6ms	remaining: 10s
3:	learn: 420905868.2557470	total: 80.8ms	remaining: 10s
4:	learn: 401531574.3361415	total: 101ms	remaining: 9.96s
5:	learn: 383131152.7785391	total: 120ms	remaining: 9.91s
6:	learn: 365589271.1391343	total: 140ms	remaining: 9.87s
7:	learn: 349068861.4131246	total: 159ms	remaining: 9.78s
8:	learn: 333353665.8969131	total: 178ms	remaining: 9.72s
9:	learn: 318620915.6381087	total: 197ms	remaining: 9.64s
10:	learn: 304557908.1900832	total: 216ms	remaining: 9.6s
11:	learn: 291303761.3545455	total: 236ms	remaining: 9.6s
12:	learn: 278832489.3182355	total: 257ms	remaining: 9.64s
13:	learn: 266852626.5550125	total: 278ms	remaining: 9.64s
14:	learn: 255752719.0782660	total: 298ms	remaining: 9.63s
15:	learn: 245057068.2244136	total: 318ms	remaining: 9.61s
16:	learn: 235040626.7538220	total: 337ms	remaining: 9.59s
17:	learn

C:\Users\meteo\.conda\envs\final\lib\site-packages\vecstack\coresk.py:382: UserWarning: This is regression task hence classification-specificparameters set to ``True`` were ignored: ``stratified``
  warnings.warn(warn_str, UserWarning)


...

0:	learn: 177835749.8168758	total: 15.8ms	remaining: 7.9s
1:	learn: 169672543.8009172	total: 32.1ms	remaining: 8s
2:	learn: 162009882.0421415	total: 48.2ms	remaining: 7.99s
3:	learn: 154709307.9204993	total: 63.9ms	remaining: 7.93s
4:	learn: 147795345.6220860	total: 79.7ms	remaining: 7.89s
5:	learn: 141200205.4377365	total: 95.9ms	remaining: 7.9s
6:	learn: 134975160.2542836	total: 112ms	remaining: 7.88s
7:	learn: 129020144.1032733	total: 128ms	remaining: 7.9s
8:	learn: 123387538.6006896	total: 144ms	remaining: 7.85s
9:	learn: 118038414.1269536	total: 160ms	remaining: 7.82s
10:	learn: 112976680.3068913	total: 175ms	remaining: 7.77s
11:	learn: 108214206.4055060	total: 190ms	remaining: 7.73s
12:	learn: 103637343.7695824	total: 206ms	remaining: 7.73s
13:	learn: 99395147.5266496	total: 222ms	remaining: 7.72s
14:	learn: 95363003.1660314	total: 238ms	remaining: 7.7s
15:	learn: 91471481.1834200	total: 255ms	remaining: 7.71s
16:	learn: 87816541.5446764	total: 271ms	remaining: 7.71s
17:	learn: 8

C:\Users\meteo\.conda\envs\final\lib\site-packages\vecstack\coresk.py:382: UserWarning: This is regression task hence classification-specificparameters set to ``True`` were ignored: ``stratified``
  warnings.warn(warn_str, UserWarning)
C:\Users\meteo\.conda\envs\final\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


0:	learn: 328821151.9989524	total: 16.1ms	remaining: 8.02s
1:	learn: 314186474.7358968	total: 32.1ms	remaining: 7.99s
2:	learn: 300284947.0742206	total: 48.5ms	remaining: 8.04s
3:	learn: 287226238.7853875	total: 65.3ms	remaining: 8.1s
4:	learn: 274894452.1621119	total: 82.1ms	remaining: 8.13s
5:	learn: 263277576.7907834	total: 98.9ms	remaining: 8.14s
6:	learn: 252133724.7798435	total: 116ms	remaining: 8.15s
7:	learn: 241567407.4259189	total: 132ms	remaining: 8.14s
8:	learn: 231540566.9417983	total: 148ms	remaining: 8.09s
9:	learn: 221954913.2508164	total: 164ms	remaining: 8.03s
10:	learn: 212933842.9605438	total: 179ms	remaining: 7.96s
11:	learn: 204739444.5671469	total: 195ms	remaining: 7.94s
12:	learn: 196712889.7277418	total: 212ms	remaining: 7.93s
13:	learn: 189320694.4523696	total: 228ms	remaining: 7.91s
14:	learn: 182173461.0183054	total: 245ms	remaining: 7.92s
15:	learn: 175437446.9927141	total: 262ms	remaining: 7.93s
16:	learn: 168977041.3757986	total: 280ms	remaining: 7.96s
17

C:\Users\meteo\.conda\envs\final\lib\site-packages\vecstack\coresk.py:382: UserWarning: This is regression task hence classification-specificparameters set to ``True`` were ignored: ``stratified``
  warnings.warn(warn_str, UserWarning)
C:\Users\meteo\.conda\envs\final\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\Users\meteo\.conda\envs\final\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\Users\meteo\.conda\envs\final\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\Users\meteo\.conda\envs\fi

0:	learn: 417222150.5234480	total: 18.2ms	remaining: 9.1s
1:	learn: 398008169.6144686	total: 34.9ms	remaining: 8.68s
2:	learn: 379792783.3289896	total: 51.9ms	remaining: 8.61s
3:	learn: 362598056.5096360	total: 69.6ms	remaining: 8.63s
4:	learn: 346165501.7250721	total: 87.1ms	remaining: 8.62s
5:	learn: 330798850.9627060	total: 104ms	remaining: 8.59s
6:	learn: 315772229.1545112	total: 121ms	remaining: 8.52s
7:	learn: 301629539.3007016	total: 139ms	remaining: 8.54s
8:	learn: 288092844.0316947	total: 156ms	remaining: 8.48s
9:	learn: 275117268.2066503	total: 172ms	remaining: 8.43s
10:	learn: 262869644.7119171	total: 188ms	remaining: 8.37s
11:	learn: 251441162.1478716	total: 204ms	remaining: 8.3s
12:	learn: 240905213.6987071	total: 221ms	remaining: 8.27s
13:	learn: 230668580.5885658	total: 237ms	remaining: 8.24s
14:	learn: 220986885.0270987	total: 254ms	remaining: 8.23s
15:	learn: 211662305.0057111	total: 272ms	remaining: 8.22s
16:	learn: 203051018.9702795	total: 290ms	remaining: 8.23s
17:	